In [ ]:
from transformers import ViTMAEConfig
from transformers import ViTForImageClassification
from dataloader import test_square_xrd_classification_dataloader
import torch
from torch import nn
import numpy as np

In [ ]:
path = './YOUR TRAINED MODEL PATH HERE'

In [ ]:
model = ViTForImageClassification.from_pretrained(path )
model.config.num_labels
model= nn.DataParallel(model) #only have this if you run the model on data parallel device
model.load_state_dict(torch.load("./TRAINED VITMAE MODEL.pth",map_location='cpu')) #change accordingly to your target device 'cpu' or 'cuda'
torch.save(model.module.state_dict(), "./test_model.pth")

In [ ]:
path = './SAVE PATH FOR YOUR VITCLASSIFICATION'
model = ViTForImageClassification.from_pretrained(path)
model.load_state_dict(torch.load("./test_model.pth",map_location='cpu'))

In [ ]:

total_sum = 0
total_len = 0

for idx, data in enumerate(test_square_xrd_classification_dataloader):
        classification = data[:, :, -1, 0]-1
        classification_gpu = torch.from_numpy(classification.long().numpy()[:,0])
        data = data[:, :, :-1, :]
        output = model(data)
        
        output = np.argmax(output.logits.detach().numpy(), axis=1)
        cache = output == classification.numpy().astype(int).squeeze()
        total_sum += sum(cache)
        total_len += len(cache)
        if idx%20 ==0:
                print(f' batch {idx}, accuracy = {total_sum/total_len}')
print(f'accuracy = {total_sum/total_len}')